<a href="https://colab.research.google.com/github/rostic7997/gemini-2.0-exp-bot-telegram/blob/main/4k_Video_Upscaler_Colab_(Real_ESRGAN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup (~1 minute)

In [1]:
import torch
assert torch.cuda.is_available(), "GPU not detected.. Please change runtime to GPU"

from PIL import Image
import cv2, os, subprocess
from tqdm import tqdm

!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN

!pip install -q torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu118
!pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python
!pip install -q -r requirements.txt
!python setup.py develop

!pip install "numpy<2"
mount_drive = False

fatal: destination path 'Real-ESRGAN' already exists and is not an empty directory.
/content/Real-ESRGAN
/usr/local/lib/python3.11/dist-packages/setuptools/__init__.py:94: _DeprecatedInstaller: setuptools.installer and fetch_build_eggs are deprecated.
!!

        ********************************************************************************
        Requirements should be satisfied by a PEP 517 installer.
        If you are using pip, you can try `pip install --use-pep517`.
        ********************************************************************************

!!
  dist.fetch_build_eggs(dist.setup_requires)
running develop
/usr/local/lib/python3.11/dist-packages/setuptools/command/develop.py:41: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer or other
        sta

In [ ]:
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext, CallbackQueryHandler
import os
import subprocess
import cv2
import torch
import time
from functools import partial

# Убедитесь, что GPU доступен
assert torch.cuda.is_available(), "GPU not detected. Please change runtime to GPU."

# Установите необходимые библиотеки
subprocess.run("pip install -q torch==2.0.1 torchvision==0.15.2 --extra-index-url https://download.pytorch.org/whl/cu118", shell=True)
subprocess.run("pip install -q basicsr facexlib gfpgan ffmpeg ffmpeg-python", shell=True)
subprocess.run("git clone https://github.com/xinntao/Real-ESRGAN.git", shell=True)

# Перейдите в директорию Real-ESRGAN
os.chdir("Real-ESRGAN")
subprocess.run("pip install -q -r requirements.txt", shell=True)
subprocess.run("python setup.py develop", shell=True)
subprocess.run("pip install 'numpy<2'", shell=True)

# Укажите токен вашего бота
TOKEN = "8118078532:AAHQUL3RK-6zZbPUx5G7x-YsuMM3vO9rLLo"

# Глобальная переменная для хранения информации о прогрессе
user_data = {}

def start(update: Update, context: CallbackContext) -> None:
    keyboard = [
        [InlineKeyboardButton("Улучшить видео", callback_data='upscale_video')],
    ]
    reply_markup = InlineKeyboardMarkup(keyboard)
    update.message.reply_text('Привет! Нажмите на кнопку ниже, чтобы улучшить видео.', reply_markup=reply_markup)

def button_click(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()
    if query.data == 'upscale_video':
        keyboard = [
            [InlineKeyboardButton("FHD (1920 x 1080)", callback_data='res_FHD')],
            [InlineKeyboardButton("2k (2560 x 1440)", callback_data='res_2k')],
            [InlineKeyboardButton("4k (3840 x 2160)", callback_data='res_4k')],
        ]
        reply_markup = InlineKeyboardMarkup(keyboard)
        query.edit_message_text(text="Пожалуйста, выберите разрешение для улучшения видео:", reply_markup=reply_markup)

def resolution_choice(update: Update, context: CallbackContext) -> None:
    query = update.callback_query
    query.answer()
    resolution = query.data.split('_')[1]
    context.user_data['resolution'] = resolution
    query.edit_message_text(text=f"Вы выбрали разрешение: {resolution}. Пожалуйста, загрузите видео для улучшения.")

def handle_video(update: Update, context: CallbackContext) -> None:
    video_file = update.message.video.get_file()
    video_path = f"/content/{update.message.video.file_id}.mp4"
    video_file.download(video_path)

    resolution = context.user_data.get('resolution', 'FHD')
    output_dir = "/content/"
    model = "RealESRGAN_x4plus"

    # Сообщите пользователю, что обработка началась
    update.message.reply_text("Обработка видео началась. Пожалуйста, подождите...")

    # Запустите обработку видео в отдельном потоке, чтобы не блокировать бота
    process_video(update, context, video_path, output_dir, resolution, model)

def process_video(update, context, video_path, output_dir, resolution, model):
    start_time = time.time()

    # Определите разрешение
    final_width, final_height = {
        "FHD": (1920, 1080),
        "2k": (2560, 1440),
        "4k": (3840, 2160)
    }.get(resolution, (1920, 1080))

    # Выполните улучшение видео
    subprocess.run(f"python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale 2", shell=True)

    # Отправьте улучшенное видео обратно пользователю
    with open(f"{output_dir}{os.path.basename(video_path)}", 'rb') as video:
        context.bot.send_video(chat_id=update.effective_chat.id, video=video)

    # Сообщите пользователю, что обработка завершена
    elapsed_time = time.time() - start_time
    update.message.reply_text(f"Обработка видео завершена за {elapsed_time:.2f} секунд.")

def main():
    updater = Updater(TOKEN)
    dispatcher = updater.dispatcher

    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(CallbackQueryHandler(button_click))
    dispatcher.add_handler(CallbackQueryHandler(resolution_choice))
    dispatcher.add_handler(MessageHandler(Filters.video, handle_video))

    updater.start_polling()
    updater.idle()

if __name__ == '__main__':
    main()



# 2. Mount drive (optional)

In [ ]:
from google.colab import drive
mount_drive=False #@param{type:"boolean"}

if mount_drive:
  drive.mount('/content/gdrive/')

# 3. Upscale video

- The upscaled video will be saved to `output_dir`
- If google drive is mounted, it will be also saved at `MyDrive/Upscaled Videos (REAL-ESRGAN)`


In [2]:
video_path="/content/video.mp4" #@param{type:"string"}
output_dir="/content/" #@param{type:"string"}
resolution = "FHD (1920 x 1080)" # @param ["FHD (1920 x 1080)", "2k (2560 x 1440)", "4k (3840 x 2160)","2 x original", "3 x original", "4 x original"] {type:"string"}
model = "RealESRGAN_x4plus" #@param ["RealESRGAN_x4plus" , "RealESRGAN_x4plus_anime_6B", "realesr-animevideov3"]

assert os.path.exists(video_path), "Video file does not exist"

video_capture = cv2.VideoCapture(video_path)
video_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

final_width = None
final_height = None
aspect_ratio = float(video_width/video_height)

# Get output resolutions
match resolution:
  case "FHD (1920 x 1080)":
    final_width=1920
    final_height=1080
  case "2k (2560 x 1440)":
    final_width=2560
    final_height=1440
  case "4k (3840 x 2160)":
    final_width=3840
    final_height=2160
  case "2 x original":
    final_width=2*video_width
    final_height=2*video_height
  case "3 x original":
    final_width=3*video_width
    final_height=3*video_height
  case "4 x original":
    final_width=4*video_width
    final_height=4*video_height

if aspect_ratio == 1.0 and "original" not in resolution:
  final_height = final_width

if aspect_ratio < 1.0 and "original" not in resolution:
  temp = final_width
  final_width = final_height
  final_height = temp

scale_factor = max(final_width/video_width, final_height/video_height)
isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

# scale_factor needs to be even
while isEven == False:
  scale_factor += 0.01
  isEven = int(video_width * scale_factor) % 2 == 0 and int(video_height * scale_factor) % 2 == 0

print(f"Upscaling from {video_width}x{video_height} to {final_width}x{final_height}, scale_factor={scale_factor}")

!python inference_realesrgan_video.py -n {model} -i '{video_path}' -o '{output_dir}' --outscale {scale_factor}

video_name_with_ext = os.path.basename(video_path)
video_name = video_name_with_ext.replace(".mp4","")
upscaled_video_path = f"{output_dir}{video_name}_out.mp4"
final_video_name = f"{video_name}_upscaled_{final_width}_{final_height}.mp4"
final_video_path = os.path.join(output_dir, final_video_name)

# crop to fit
if "original" not in resolution:
  print("Cropping to fit...")
  command = f"ffmpeg -loglevel error -hwaccel cuda -y -i '{upscaled_video_path}' -c:v h264_nvenc -filter:v  'crop={final_width}:{final_height}:(in_w-{final_width})/2:(in_h-{final_height})/2' -c:v libx264 -pix_fmt yuv420p '{final_video_path}'"
  subprocess.run(command,shell=True)
else:
  # final video path = upscaled video path
  command = f"cp '{upscaled_video_path}' '{final_video_path}'"
  subprocess.run(command,shell=True)

print(f"Upscaled video saved to: {final_video_path}")

# save to drive
if mount_drive:
  drive_folder = "MyDrive/Upscaled Videos (REAL-ESRGAN)"
  save_directory_drive = f"/content/gdrive/{drive_folder}"
  os.makedirs(save_directory_drive, exist_ok=True)

  command = f"cp '{final_video_path}' '{save_directory_drive}/{final_video_name}'"
  subprocess.run(command,shell=True)
  print(f"Saved to drive: /{drive_folder}/{final_video_name}" )

!rm "{upscaled_video_path}"

Upscaling from 360x640 to 1080x1920, scale_factor=3.0
/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
Downloading: "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" to /content/Real-ESRGAN/weights/RealESRGAN_x4plus.pth

100% 63.9M/63.9M [00:00<00:00, 293MB/s]
inference: 100% 210/210 [05:26<00:00,  1.56s/frame]
Cropping to fit...
Upscaled video saved to: /content/video_upscaled_1080_1920.mp4


# 4. Disconnect runtime

In [3]:
from google.colab import runtime

disconnect_when_finish = False  #@param{type:"boolean"}

if disconnect_when_finish:
  runtime.unassign()